In [2]:
import numpy as np
from numpy import linalg

from qiskit.providers.basicaer import QasmSimulatorPy
from qiskit.providers.aer import QasmSimulator
from qiskit import Aer
from qiskit.algorithms.optimizers import SPSA, COBYLA

from qiskit.opflow import X, Y, Z, I
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info.operators import Operator, Pauli

In [18]:
#optimizer = COBYLA(maxiter=50, tol=0.001, rhobeg=1.0)
optimizer = SPSA(maxiter=100)
    
backend = Aer.get_backend("qasm_simulator")
quantum_instance = QuantumInstance(backend, shots=8192)

intermediate_info = {
    'nfev': [],
    'parameters': [],
    'energy': [],
    'stddev': []
}

def callback(nfev, parameters, energy, stddev):
    intermediate_info['nfev'].append(nfev)
    intermediate_info['parameters'].append(parameters)
    intermediate_info['energy'].append(energy)
    intermediate_info['stddev'].append(stddev)
    
def Classical_GS_energy(H):
    E_l,P_l = linalg.eig(Operator(H).data)
    Egs = np.min(E_l)
    return Egs

def Quantum_GS_energy(H):
    
    # ricavo numero siti direttamente da hamiltoniana
    Nq = int(np.sqrt(np.shape(Operator(H).data)[1]))
    
    # stima del GS con EfficientSU(2)
    ansatz = EfficientSU2(Nq, reps=0, entanglement='linear', insert_barriers=True) 
    np.random.seed(5)  # seed for reproducibility
    initial_point = np.random.random(ansatz.num_parameters)
    local_vqe = VQE(ansatz=ansatz,
                    optimizer=optimizer,
                    initial_point=initial_point,
                    quantum_instance=QasmSimulatorPy(),
                    callback=callback)

    local_result = local_vqe.compute_minimum_eigenvalue(H)
    return local_result.eigenvalue

In [19]:
# Ising model parameters
N = 4
J = 1
h = 1

# hamiltonian
H = J*(X^X^I^I) + J*(I^X^X^I) + J*(I^I^X^X) + J*(X^I^I^X) + h*(Z^I^I^I) + h*(I^Z^I^I) + h*(I^I^Z^I) + h*(I^I^I^Z)

print('Quantum GS energy  : ', Quantum_GS_energy(H))
print('Classical GS energy: ', Classical_GS_energy(H))

Quantum GS energy  :  (-4.662109375+0j)
Classical GS energy:  (-5.22625185950551+0j)


In [ ]:
# work in progress: given N sites write hamiltonian

from qiskit.quantum_info.operators import Operator, Pauli

X = Operator(Pauli('X'))
Z = Operator(Pauli('Z'))
Y = Operator(Pauli('Y'))
I = Operator(Pauli('I'))


# metodo diretto, sbagliato
N=2
M=1
H = np.zeros((2**N,2**N), dtype='complex')
i=0
for k in range(N-1):
    A=1
    B=1
    while i<N:
        if i==k:
            A = np.kron(np.kron(A,Z),Z)
            B = np.kron(B,Z)
            B = np.kron(B,I)
            i += 2
        else:
            A = np.kron(A,I)
            B = np.kron(B,I)
            i+=1
        #print(i, np.shape(B)
        print(A)
    i = 0
    print(k)
    H = H + A

In [ ]:
# metodo funzionante per N>2 ma inefficiente

N=4
H = np.zeros((2**N,2**N), dtype='complex')

for i in range(N-1):
    k=0
    lista1 = []
    lista2 = []
    while k<N:
        if k==i:
            lista1.append(X)
            lista1.append(X)
            lista2.append(Z)
            lista2.append(I)
            k += 2
        else:
            lista1.append(I)
            lista2.append(I)
            k += 1
    #print(i, len(lista1),len(lista2))
    A=1
    B=1
    for j in range(N):
        A = np.kron(A,lista1[j])
        B = np.kron(B,lista2[j])
    #print(i, np.shape(A))
    H = H + A + B
    
lista1 = []
lista2 = []

#PBC
k = 0
while k<N:
        if k==0:
            lista1.append(X)
            lista2.append(I)
            k += 1
        elif k==N-1:
            lista1.append(X)
            lista2.append(Z)
            k += 1
        else:
            lista1.append(I)
            lista2.append(I)
            k += 1
A=1
B=1
for j in range(N):
    A = np.kron(A,lista1[j])
    B = np.kron(B,lista2[j])
H = H + A + B